#### duplicate file for learning 

In [1573]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [1574]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


In [1575]:
# Open Sheet
sheet = client.open_by_key("1vadc79ujOS66vVhIhRlieVafQKMBbaixvvSNFU0mJRQ") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Sheets - Managed by GCPL MIS Exec

In [1576]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
MSLMaster = sheet.worksheet("MSLMaster")

In [1577]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
MSLMaster = get_as_dataframe(MSLMaster)

In [1578]:
# SOSTargetMaster.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\StoreMaster.csv', index=False)

In [1579]:
SOSTargetMaster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Region                  50 non-null     object 
 1   Chain Code              50 non-null     object 
 2   Chain                   50 non-null     object 
 3   Category Tracking Code  50 non-null     object 
 4   LIS Category            50 non-null     object 
 5   SOS Target              50 non-null     float64
 6   Start Date              50 non-null     object 
dtypes: float64(1), object(6)
memory usage: 3.1+ KB


#### Connection with SQL Server ( Optional for Now) once SQL database corrected , will be connected to Server 

In [1580]:
# server = '157.119.230.120,4071'    
# database = 'PowerBI_LISV2'      
# username = 'sa'
# password = 'Y0m@Sql.!.123'

# # Connection string
# conn_str = f"""
#     DRIVER={{ODBC Driver 17 for SQL Server}};
#     SERVER={server};
#     DATABASE={database};
#     UID={username};
#     PWD={password};
#     TrustServerCertificate=yes;
# """

## Data Collection

### CategoryTracking

In [1581]:
# CategoryTrackingAIReportExport = pd.read_sql(query, conn)
# conn.close()

#### CSV CategoryTrackingAI

In [1582]:
### Table Name : CTReportExportAI_26-05-2025.xls

In [1583]:
# CSV Read Module
CTReportExportAI = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTAIReportExport_19-06-2025.xlsx")
# CTReportExportAI= pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_21-05-2025.xls')
CTReportExportAI.rename(columns= {'Submitted Date&Time':'Date','Sync Date&Time':'SyncDate&Time','LISStoreCode ':'LISStoreCode','Category Tracking Code':'CategoryTrackingCode','Chain Name':'ChainName',
                                  'Reason for Non availability' :'Non Availability Reason','AI detection reason' : 'AIdetectionreason',
                                 'AI Detection Status': 'AIDetectionStatus'}, inplace=True)

# SQL Based filters
CTReportExportAI_1 = CTReportExportAI[
    (
        CTReportExportAI['AIDetectionStatus'].isin(['Yes', 'No']) | 
        CTReportExportAI['AIDetectionStatus'].isna()
    ) &
    CTReportExportAI['CategoryTrackingCode'].isin([
        'AERAEROSOL-01', 'Aerosol-01', 'AERPocket-01',
        'HAIRCOLOR-01', 'HITAEROSOL-01', 'LIQMACHREF-01',
        'LIQUIDDETERGENT-01', 'Liquids-01', 'SOAPS-01'
    ])
]

CategoryTrackingAIReportExport=CTReportExportAI_1


In [1584]:
CategoryTrackingAIReportExport=CategoryTrackingAIReportExport.drop(columns=['SyncDate&Time', 'Month', 'Year'])

In [1585]:
# CategoryTrackingAIReportExport['AuditQuestions'].unique

In [1586]:
CategoryTrackingAIReportExport.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19666 entries, 1 to 41681
Data columns (total 58 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Visit ID                                              19666 non-null  int64  
 1   Date                                                  19666 non-null  object 
 2   Region                                                19666 non-null  object 
 3   State                                                 19666 non-null  object 
 4   City                                                  19666 non-null  object 
 5   RSMCode                                               0 non-null      float64
 6   BranchCode                                            0 non-null      float64
 7   ASMCode                                               15329 non-null  object 
 8   SOCode                                                0 non-n

In [1587]:
# GCPL_StoreMaster1 = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252')

In [1588]:
# GCPL_StoreMaster = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252', 
#                                usecols = ['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name'])

In [1589]:
GCPL_StoreMaster= StoreMaster[['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name']]

In [1590]:
GCPL_StoreMaster.head(2)

LIS Store Code Supervisor Name Store Status  RKAM          ASM      SO Name
0   GCPL-SAMT-07       David Raj       Active   NaN  Karthikeyan  Karthikeyan
1  GCPL-SAMT-999       David Raj       Active   NaN  Karthikeyan  Karthikeyan

In [1591]:
CategoryTrackingAIReportExport2 = pd.merge(CategoryTrackingAIReportExport,GCPL_StoreMaster,how='left',left_on='LIS Store Code',
                                           right_on='LIS Store Code')

# LIS Store Code NULL wala condition to be added

In [1592]:
CategoryTrackingAIReportExport2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19666 entries, 0 to 19665
Data columns (total 63 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Visit ID                                              19666 non-null  int64  
 1   Date                                                  19666 non-null  object 
 2   Region                                                19666 non-null  object 
 3   State                                                 19666 non-null  object 
 4   City                                                  19666 non-null  object 
 5   RSMCode                                               0 non-null      float64
 6   BranchCode                                            0 non-null      float64
 7   ASMCode                                               15329 non-null  object 
 8   SOCode                                                0 

In [1593]:
# MasterSOSTarget = pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\Master_SOSTarget.xlsx', usecols = ['Category Tracking Code','Start Date'])

In [1594]:
MasterSOSTarget = SOSTargetMaster[['Chain Code','Category Tracking Code','Start Date','SOS Target']]

In [1595]:
MasterSOSTarget.head()

Chain Code Category Tracking Code Start Date  SOS Target
0  SAMT-KN-ISP-01          AERAEROSOL-01  01-Sep-24         0.2
1  SAMT-KN-ISP-01           AERPocket-01  01-Sep-24         0.5
2  SAMT-KN-ISP-01           HAIRCOLOR-01  01-Sep-24         0.2
3  SAMT-KN-ISP-01          HITAEROSOL-01  01-Sep-24         0.7
4  SAMT-KN-ISP-01          LIQMACHREF-01  01-Sep-24         0.7

In [1596]:
CategoryTrackingAIReportExport3 = pd.merge(
    CategoryTrackingAIReportExport2,
    MasterSOSTarget,
    how='left',
    left_on=['Chain Code', 'CategoryTrackingCode'],
    right_on=['Chain Code', 'Category Tracking Code']
)

In [1597]:
# pd.to_datetime(your_series, dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Date'],dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['Start Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Start Date'],dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['CatStartCheck'] = CategoryTrackingAIReportExport3.apply(lambda row: 1 if row['Date'] >= row['Start Date'] else 0,axis=1)

In [1598]:
CategoryTrackingAIReportExport3_filtered = CategoryTrackingAIReportExport3[
    (CategoryTrackingAIReportExport3['CatStartCheck'] == 1) &
    (~CategoryTrackingAIReportExport3['ChainName'].isin(['Cp Wholesale', 'D Mart', 'Metro C&C', 'Walmart C&C']))
]
# can also be done in SQL 

In [1599]:
CategoryTrackingAIReportExport3_filtered=CategoryTrackingAIReportExport3_filtered.fillna(0)

In [1600]:
df = CategoryTrackingAIReportExport3_filtered
df = df[df['Store Status'] == 'Active']


In [1601]:
#test 80392
dftest = df[df['Visit ID']== 80392 ]
dftest[['Visit ID','AIdetectionreason']].head()

Empty DataFrame
Columns: [Visit ID, AIdetectionreason]
Index: []

In [1602]:
df.rename(columns={"Non Availability Reason": "Non Availability Reason-old"}, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\2723549263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Non Availability Reason": "Non Availability Reason-old"}, inplace=True)


In [1603]:
# df.info()?

In [1604]:
df['Visit ID'].count()

19536

In [1605]:
# df['Non Availability Reason'] = df['Non Availability Reason-old'].combine_first(df['AIdetectionreason'])
df['Non Availability Reason'] = df.apply(
    lambda row: row['AIdetectionreason'] if row['AIDetectionStatus'] == 'No' else row['Non Availability Reason-old'],
    axis=1
)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\2149626607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Non Availability Reason'] = df.apply(


In [1606]:
df['Visit ID'].count()

19536

In [1607]:
df['Key'] = df['Date'].dt.strftime('%d/%m/%Y') + df['LIS Store Code'] + df['Category Tracking Code'].astype(str)
# df = df.drop(columns=["Non Availability Reason-old", "Start Date", "CatStartCheck"])

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\2215685716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Date'].dt.strftime('%d/%m/%Y') + df['LIS Store Code'] + df['Category Tracking Code'].astype(str)


In [1608]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\test.csv', index=False)

In [1609]:
df.head()

Visit ID                Date Region      State       City  RSMCode  \
0     83281 2025-06-18 16:39:38  South  Karnataka  Bangalore      0.0   
1     83281 2025-06-18 16:39:38  South  Karnataka  Bangalore      0.0   
2     83281 2025-06-18 16:39:38  South  Karnataka  Bangalore      0.0   
3     83281 2025-06-18 16:39:38  South  Karnataka  Bangalore      0.0   
4     83281 2025-06-18 16:39:38  South  Karnataka  Bangalore      0.0   

   BranchCode ASMCode  SOCode SupervisorCode  ... Store Status RKAM      ASM  \
0         0.0       0     0.0    GCPL-SUP-01  ...       Active  0.0  Vignesh   
1         0.0       0     0.0    GCPL-SUP-01  ...       Active  0.0  Vignesh   
2         0.0       0     0.0    GCPL-SUP-01  ...       Active  0.0  Vignesh   
3         0.0       0     0.0    GCPL-SUP-01  ...       Active  0.0  Vignesh   
4         0.0       0     0.0    GCPL-SUP-01  ...       Active  0.0  Vignesh   

       SO Name Category Tracking Code Start Date SOS Target CatStartCheck  \
0  Karthikeyan          AERAEROSOL-01 2024-09-01        0.2             1   
1  Karthikeyan           AERPocket-01 2024-09-01        0.5             1   
2  Karthikeyan           HAIRCOLOR-01 2024-09-01        0.2             1   
3  Karthikeyan          HITAEROSOL-01 2024-09-01        0.7             1   
4  Karthikeyan          LIQMACHREF-01 2024-09-01        0.7             1   

  Non Availability Reason                                   Key  
0                       0  18/06/2025GCPL-SAMT-830AERAEROSOL-01  
1                       0   18/06/2025GCPL-SAMT-830AERPocket-01  
2                       0   18/06/2025GCPL-SAMT-830HAIRCOLOR-01  
3                       0  18/06/2025GCPL-SAMT-830HITAEROSOL-01  
4                       0  18/06/2025GCPL-SAMT-830LIQMACHREF-01  

[5 rows x 69 columns]

In [1610]:
df = df.drop_duplicates(subset='Key').reset_index(drop=True)

In [1611]:
df['Visit ID'].count()

19331

In [1612]:

df['Format2'] = 'SAMT'
df['ImagePath'] =''
df['Category Type']='Format'
df['SOS TGT']=''
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['ChainName'] = df['ChainName'].str.replace(r'^SAMT\s+', '', regex=True)
df['SOSAch%_']= df['Facing Contribution']


In [1613]:
df['AISOSAch%_'] = (
    (df['Self Product Facings Detected'] / df['Total Facings']).replace([np.inf, -np.inf], 0).fillna(0) * 100
).round(0).clip(upper=100)

In [1614]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19331 entries, 0 to 19330
Data columns (total 77 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Visit ID                                              19331 non-null  int64         
 1   Date                                                  19331 non-null  datetime64[ns]
 2   Region                                                19331 non-null  object        
 3   State                                                 19331 non-null  object        
 4   City                                                  19331 non-null  object        
 5   RSMCode                                               19331 non-null  float64       
 6   BranchCode                                            19331 non-null  float64       
 7   ASMCode                                               19331 non-null  object

In [1615]:
### Columns ReOrdering
df=df[['Visit ID','Date','Region','State','RKAM','ASMCode','SO Name','SupervisorCode','Supervisor Name','UserName','FullName','Month','Year','Format2','Chain Code','ChainName','LIS Store Code','Store Name','Is category Available','Non Availability Reason','AIDetectionStatus','Category Tracking Name','Final Image Path','Annotated Image Path','ImagePath','Category Type','Tgt Cat SOS','Facings','Total Facings','Facing Contribution','SOSAch%_','Self Product Facings Detected','Competition Products Detected','Total Facings Detected','AISOSAch%_']]


In [1616]:
# df['SOSAch%_']= df['Facing Contribution']

In [1617]:
df['Visit ID'].count()

19331

In [1618]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\AICAtegoryTrackingSOS_ModelOutput.csv', index=False)

#### End of SOS

In [1619]:
df_VisitID = pd.DataFrame(df['Visit ID'].unique(), columns=['Visit ID'])

In [1620]:
df_VisitID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255 entries, 0 to 2254
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Visit ID  2255 non-null   int64
dtypes: int64(1)
memory usage: 17.7 KB


## Primary ShelfStock

In [1537]:
PrimaryShelfStock = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTSelfProductAvailabilityAI_19-06-2025.xlsx")

In [1538]:
PrimaryShelfStock=PrimaryShelfStock.drop(columns=['Sync Date&Time', 'Month','Year','RSMCode','BranchCode','SOCode',
                                                  'PJP Status','Store Club','Sub Chain Name','Store Address','Store Type','Distributor Code',
                                                 'Distributor Name'])

In [1539]:
PrimaryShelfStocktest = PrimaryShelfStock[PrimaryShelfStock['Visit ID']==81766]
pd.set_option('display.max_colwidth', None) 
PrimaryShelfStocktest.head()


Visit ID  Submitted Date&Time Region       State   City  \
88969     81766  07-06-2025 11:19:54  South  Tamil Nadu  Salem   
88970     81766  07-06-2025 11:19:54  South  Tamil Nadu  Salem   
88971     81766  07-06-2025 11:19:54  South  Tamil Nadu  Salem   
88972     81766  07-06-2025 11:19:54  South  Tamil Nadu  Salem   
88973     81766  07-06-2025 11:19:54  South  Tamil Nadu  Salem   

              ASMCode SupervisorCode         ISPCode        UserName  \
88969  GCPLSAMT-FO-05    GCPL-SUP-10  GCPLSAMT-MR-39  GCPLSAMT-MR-39   
88970  GCPLSAMT-FO-05    GCPL-SUP-10  GCPLSAMT-MR-39  GCPLSAMT-MR-39   
88971  GCPLSAMT-FO-05    GCPL-SUP-10  GCPLSAMT-MR-39  GCPLSAMT-MR-39   
88972  GCPLSAMT-FO-05    GCPL-SUP-10  GCPLSAMT-MR-39  GCPLSAMT-MR-39   
88973  GCPLSAMT-FO-05    GCPL-SUP-10  GCPLSAMT-MR-39  GCPLSAMT-MR-39   

           FullName  ... AI Detection Status Detection Reason Product Type  \
88969  SILAMBARASAN  ...                 Yes              NaN         Self   
88970  SILAMBARASAN  ...                 Yes              NaN         Self   
88971  SILAMBARASAN  ...                 Yes              NaN         Self   
88972  SILAMBARASAN  ...                 Yes              NaN         Self   
88973  SILAMBARASAN  ...                 Yes              NaN         Self   

          AI SKU Code                         Product Name Product Category  \
88969   GCPL-AI-ACCMA                             Cin_Cool    Personal Wash   
88970  GCPL-AI-ACCO5M  Cin_Cool_100G_B4G1_M224_MCRT_P30_MT    Personal Wash   
88971   GCPL-AI-ACSM2  Cin_Helth_Plus_125G_PO3_M186_P40_MT    Personal Wash   
88972   GCPL-AI-ACFMA                             Cin_Lime    Personal Wash   
88973  GCPL-AI-ACRM95    Cin_Lime_150G_PO5_MT_M304_P12_New    Personal Wash   

      Product Brand Facings Detected  \
88969       CINTHOL              3.0   
88970       Cinthol              2.0   
88971       Cinthol              1.0   
88972       Cinthol              4.0   
88973       CINTHOL              2.0   

                                                                                      Final Image Path  \
88969  https://lisimagev2.loveinstore.com//imagepath/detected_504f379e-bb14-4ccc-88fa-8721944d8682.jpg   
88970  https://lisimagev2.loveinstore.com//imagepath/detected_504f379e-bb14-4ccc-88fa-8721944d8682.jpg   
88971  https://lisimagev2.loveinstore.com//imagepath/detected_504f379e-bb14-4ccc-88fa-8721944d8682.jpg   
88972  https://lisimagev2.loveinstore.com//imagepath/detected_504f379e-bb14-4ccc-88fa-8721944d8682.jpg   
88973  https://lisimagev2.loveinstore.com//imagepath/detected_504f379e-bb14-4ccc-88fa-8721944d8682.jpg   

                                                                                  Annotated Image Path  
88969  https://lisimagev2.loveinstore.com//imagepath/detected_c73764e1-628f-41d8-a778-a9f98f1d45e5.png  
88970  https://lisimagev2.loveinstore.com//imagepath/detected_c73764e1-628f-41d8-a778-a9f98f1d45e5.png  
88971  https://lisimagev2.loveinstore.com//imagepath/detected_c73764e1-628f-41d8-a778-a9f98f1d45e5.png  
88972  https://lisimagev2.loveinstore.com//imagepath/detected_c73764e1-628f-41d8-a778-a9f98f1d45e5.png  
88973  https://lisimagev2.loveinstore.com//imagepath/detected_c73764e1-628f-41d8-a778-a9f98f1d45e5.png  

[5 rows x 34 columns]

In [1540]:
PrimaryShelfStock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133557 entries, 0 to 133556
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Visit ID                     133557 non-null  int64  
 1   Submitted Date&Time          133557 non-null  object 
 2   Region                       133557 non-null  object 
 3   State                        133557 non-null  object 
 4   City                         133557 non-null  object 
 5   ASMCode                      103109 non-null  object 
 6   SupervisorCode               132637 non-null  object 
 7   ISPCode                      132637 non-null  object 
 8   UserName                     133557 non-null  object 
 9   FullName                     133557 non-null  object 
 10  Designation                  133557 non-null  object 
 11  Client Store Code            132008 non-null  object 
 12  LIS Store Code               133557 non-null  object 
 13 

In [1541]:
# to verify all the available visit ids
PrimaryShelfStock['is_check'] = PrimaryShelfStock['Visit ID'].isin(df_VisitID['Visit ID']).astype(int)

In [1542]:
PrimaryShelfStock = PrimaryShelfStock[PrimaryShelfStock['is_check']==1]

In [1543]:
print(PrimaryShelfStock['Visit ID'].count())

132621


In [1544]:
# filtering null visit id and produtc type only toself
PrimaryShelfStock2 = PrimaryShelfStock[(PrimaryShelfStock['Visit ID'].notna()) & (PrimaryShelfStock['Product Type'] == 'Self')]

In [1545]:
# PrimaryShelfStock2 = PrimaryShelfStock2.drop(columns=["Sync Date&Time", "Month", "Year"], errors='ignore')
PrimaryShelfStock2 = PrimaryShelfStock2.rename(columns={"Submitted Date&Time": "Date"})

In [1546]:
PrimaryShelfStock2['Facings Detected Check'] = PrimaryShelfStock2['Facings Detected'].apply(lambda x: 1 if x > 0 else 0)

#AI Availability column


In [1547]:
print(PrimaryShelfStock2['Visit ID'].count())

129119


In [1548]:
MSLMaster['key'] = MSLMaster["Chain Code"]+MSLMaster["Category Tracking Code"]+MSLMaster["Roboflow Name"]

In [1549]:
MSLMaster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 303
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category Tracking Code  304 non-null    object
 1   Category Tracking Name  304 non-null    object
 2   Category Product Code   304 non-null    object
 3   Category Product Name   304 non-null    object
 4   Product Category        304 non-null    object
 5   Product Brand           304 non-null    object
 6   Product ID              304 non-null    object
 7   Product Desc            304 non-null    object
 8   AI_SKU_CODE             304 non-null    object
 9   Roboflow Name           304 non-null    object
 10  Chain Code              304 non-null    object
 11  Chain Name              304 non-null    object
 12  MSL                     304 non-null    object
 13  key                     304 non-null    object
dtypes: object(14)
memory usage: 35.6+ KB


In [1550]:
MSLMaster_renamed = MSLMaster.rename(columns={"Roboflow Name": "Product Name"})

PrimaryShelfStock3 = pd.merge(PrimaryShelfStock2,
    MSLMaster_renamed,
    on=["Chain Code", "Category Tracking Code", "Product Name"],
    how="left"
)

In [1551]:
print(PrimaryShelfStock3['Visit ID'].count())

129119


In [1552]:
PrimaryShelfStock3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129119 entries, 0 to 129118
Data columns (total 47 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Visit ID                     129119 non-null  int64  
 1   Date                         129119 non-null  object 
 2   Region                       129119 non-null  object 
 3   State                        129119 non-null  object 
 4   City                         129119 non-null  object 
 5   ASMCode                      99728 non-null   object 
 6   SupervisorCode               128257 non-null  object 
 7   ISPCode                      128257 non-null  object 
 8   UserName                     129119 non-null  object 
 9   FullName                     129119 non-null  object 
 10  Designation                  129119 non-null  object 
 11  Client Store Code            127589 non-null  object 
 12  LIS Store Code               129119 non-null  object 
 13 

In [1553]:
PrimaryShelfStock4 = PrimaryShelfStock3[PrimaryShelfStock3['MSL'] == 'Yes'] # MSL Fltered Data

In [1554]:
print(PrimaryShelfStock3['Visit ID'].count())
print(PrimaryShelfStock4['Visit ID'].count())

129119
55201


#### SKU Master

In [1555]:
SKUMaster = pd.merge(
    StoreMaster, MSLMaster,left_on="Chain Code",right_on="Chain Code",how="left",suffixes=('', '_Master_MSLTarget')
)

In [1556]:
SKUMaster.head(1)

Region       State     City Distributor Code        Distributor  \
0  South  Tamil Nadu  Chennai   SAMT-TN-ISP-01  ANAND ENTERPRISES   

       Chain Code   Chain Format Store Type   Client Store Code  ...  \
0  SAMT-TN-ISP-01  TN ISP   SAMT         GT  400000-339-0399432  ...   

  Category Product Name Product Category Product Brand Product ID  \
0              Bodywash         BodyWash       Cinthol     MCBWO2   

                            Product Desc     AI_SKU_CODE  \
0  Cin Org Bdywash Bottle 200ml M120 P30  GCPL-AI-MCBWO2   

                   Roboflow Name Chain Name  MSL  \
0  Cin_Org_Bodywash_Bottle_200ml     TN ISP  Yes   

                                                      key  
0  SAMT-TN-ISP-01Bodywash-01Cin_Org_Bodywash_Bottle_200ml  

[1 rows x 40 columns]

In [1557]:
MSL = PrimaryShelfStock4[['Visit ID','Date','LIS Store Code']]

In [1558]:
MSL=MSL.drop_duplicates()

In [1559]:
MSL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2221 entries, 2 to 129064
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Visit ID        2221 non-null   int64 
 1   Date            2221 non-null   object
 2   LIS Store Code  2221 non-null   object
dtypes: int64(1), object(2)
memory usage: 69.4+ KB


In [1560]:
MSL_SKUMaster = pd.merge(MSL,SKUMaster,on="LIS Store Code",how="left",suffixes=('', '_SKU_Master'))

In [1561]:
MSL_SKUMaster.info() # 147546

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112006 entries, 0 to 112005
Data columns (total 42 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Visit ID                112006 non-null  int64  
 1   Date                    112006 non-null  object 
 2   LIS Store Code          112006 non-null  object 
 3   Region                  112006 non-null  object 
 4   State                   112006 non-null  object 
 5   City                    112006 non-null  object 
 6   Distributor Code        112006 non-null  object 
 7   Distributor             112006 non-null  object 
 8   Chain Code              112006 non-null  object 
 9   Chain                   112006 non-null  object 
 10  Format                  112006 non-null  object 
 11  Store Type              112006 non-null  object 
 12  Client Store Code       112006 non-null  object 
 13  Store Name              112006 non-null  object 
 14  Address             

In [1562]:
MSL_SKUMaster=MSL_SKUMaster.drop(columns=['RKAM', 'Week Off', 'Sales Target','Supervisor Code','SO Code','ZM'])

In [1563]:
PrimaryShelfStock3_1 = PrimaryShelfStock3[["Date", "LIS Store Code", "Product Name",'Final Image Path','Annotated Image Path','Facings Detected Check']]

In [1564]:
MSL_SKUMaster2 = pd.merge(
    MSL_SKUMaster,
    PrimaryShelfStock3_1,
    left_on=["Date", "LIS Store Code", "Roboflow Name"],
    right_on=["Date", "LIS Store Code", "Product Name"],
    how="left",
    suffixes=('', '_PrimaryShelf')
)

In [1565]:
MSL_SKUMaster3 = pd.merge(
    MSL_SKUMaster2,
    SOSTargetMaster,
    left_on=["Category Tracking Code","Chain Code"],
    right_on=["Category Tracking Code","Chain Code"],
    how="left",
    suffixes=('', '_Master_SOSTarget')
)

In [1566]:
# few changes as per format
MSL_SKUMaster3['Format']='MT'
MSL_SKUMaster3['ManualAvailability']=''

MSL_SKUMaster3['Date'] = pd.to_datetime(MSL_SKUMaster3['Date'], errors='coerce')
MSL_SKUMaster3['Day'] = MSL_SKUMaster3['Date'].dt.day

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\4232063922.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  MSL_SKUMaster3['Date'] = pd.to_datetime(MSL_SKUMaster3['Date'], errors='coerce')


In [1567]:
MSL_SKUMaster3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112029 entries, 0 to 112028
Data columns (total 47 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Visit ID                 112029 non-null  int64         
 1   Date                     112029 non-null  datetime64[ns]
 2   LIS Store Code           112029 non-null  object        
 3   Region                   112029 non-null  object        
 4   State                    112029 non-null  object        
 5   City                     112029 non-null  object        
 6   Distributor Code         112029 non-null  object        
 7   Distributor              112029 non-null  object        
 8   Chain Code               112029 non-null  object        
 9   Chain                    112029 non-null  object        
 10  Format                   112029 non-null  object        
 11  Store Type               112029 non-null  object        
 12  Client Store Cod

In [1568]:
## ordering col

In [1569]:
MSLFinal = MSL_SKUMaster3[['Visit ID','Date','Region','State','City','ASM','SO Name','Supervisor Name','Emp User ID','Employee Name','Chain','LIS Store Code',
                           'Store Name','Category Tracking Name','Product Category','Product ID','AI_SKU_CODE',
                           'Product Desc','Facings Detected Check','ManualAvailability',
                           'Final Image Path','Annotated Image Path','MSL','Format','Day','Start Date']]

In [1570]:
MSLFinal['Date'] = pd.to_datetime(MSLFinal['Date'])
MSLFinal['Start Date'] = pd.to_datetime(MSLFinal['Start Date'])
MSLFinal['Cat Start Date Check'] = MSLFinal.apply(
    lambda row: 1 if row['Date'] > row['Start Date'] else 0,
    axis=1
)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\1580634438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MSLFinal['Date'] = pd.to_datetime(MSLFinal['Date'])
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\1580634438.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  MSLFinal['Start Date'] = pd.to_datetime(MSLFinal['Start Date'])
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_13748\1580634438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [1571]:
MSLFinal.head(1)

Visit ID                Date Region       State         City          ASM  \
0     83272 2025-06-18 16:05:55  South  Tamil Nadu  Tirunelveli  Bharathbabu   

           SO Name Supervisor Name      Emp User ID     Employee Name  ...  \
0  Mohammed Salman       Saravanan  GCPLSAMT-ISP-85  Thanga Rajeseari  ...   

                            Product Desc Facings Detected Check  \
0  Cin Org Bdywash Bottle 200ml M120 P30                    1.0   

  ManualAvailability  \
0                      

                                                                                         Final Image Path  \
0  https://lisimagev2.loveinstore.com//LISImages/6_2025/LIS216/CategoryTrackingV2/31396_1750242740720.jpg   

                                                                              Annotated Image Path  \
0  https://lisimagev2.loveinstore.com//imagepath/detected_ef90b242-dd91-49ee-bd2f-b65529a45fc7.png   

   MSL Format Day  Start Date Cat Start Date Check  
0  Yes     MT  18         NaT                    0  

[1 rows x 27 columns]

In [1572]:
MSLFinal.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\MSLFinal.csv', index=False)